In [ ]:
"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

In [1]:
# Change dir to get SQL connector
import os
os.chdir("..")
from src.sqldb import SQLDatabase

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re

# Updating Tables

In [3]:
# Constants
URL = "https://www.mtgtop8.com/format?f=ST"

In [4]:
from src import models as m
def update(URL):
    updater = m.Updater()
    updater.update_events(URL)
    updater.update_decks_and_players()
    updater.update_deck_lists()

In [5]:
update(URL)

Updating next page...


In [6]:
updater = m.Updater()
updater.update_cards()

# Cleaning

Loading data objects...

In [7]:
with SQLDatabase() as sql_db:
    event_table = sql_db.get_dataframe_from("event")
    deck_table = sql_db.get_dataframe_from("deck")
    deck_list_table = sql_db.get_dataframe_from("deckList")
    card_table = sql_db.get_dataframe_from("card")
    pilot_table = sql_db.get_dataframe_from("pilot")

In [8]:
# Clean Dates
event_table["date"] = pd.to_datetime(event_table["date"], dayfirst=True)

# Renaming ID Columns
event_table.rename(columns={"id": "eventId"}, inplace=True)
deck_table.rename(columns={"id": "deckId"}, inplace=True)
pilot_table.rename(columns={"id": "pilotId"}, inplace=True)

# Adding easier ID column to card_table
card_table["setNumber"] = card_table["setNumber"].str.zfill(3)
card_table["cardId"] = card_table["setNumber"] + card_table["setName"]

In [9]:
deck_table.loc[deck_table.deckId==601, "name"] = "GW Adventures"
deck_table.loc[deck_table.deckId==930, "name"] = "Izzet Mill"
deck_table.loc[deck_table.deckId==931, "name"] = "Sultai Ramp"
deck_table.loc[deck_table.deckId==932, "name"] = "Orzhov Aristocrats"
deck_table.loc[deck_table.deckId==934, "name"] = "Temur Control"

Adding archetypes to Deck Table.

These are important to see how play styles are distributed across standard.

Most general archetypes should be on top, so that the more specific archetypes get assigned if applicable.

In [10]:
# Adding archetypes to deck_table
archetypes = {
    "jund": ["jund"],
    "golgari": ["golgari"],
    "aggro": ["aggro", "red deck wins", "weenie", "ww", "mono green"],
    "control": ["control", "doom", "4cc", "yorion"],
    "midrange": ["midrange", "mid range"],
    "gyruda": ["gyruda"],
    "winota": ["winota"],
    "mill": ["mill"],
    "ramp": ["ramp", "omnath", "ultimatum", "forsaken monument"],
    "rogue": ["rogue"],
    "flash": ["flash"],
    "adventure": ["adventure"],
    "food": ["food"],
    "reclamation": ["reclamation"],
    "sacrifce": ["sacrifice", "aristocrats"],
    "cycling": ["cycling"],
    "kicker": ["kicker", "kick"],
    "mutate": ["mutate"],
    "stompy": ["stompy"]
}

for archetype in archetypes:
    for placeholder in archetypes[archetype]:
        deck_table.loc[
            (deck_table.name.str.lower().str.contains(placeholder)),
            "archetype"
        ] = archetype

deck_table.loc[deck_table.name == "5c Sanctum", "archetype"] = "control"

Adding color categories to Deck Table.

Color category is a big factor when looking at how the makeup of standard has evolved.

In [11]:
# Adding easy color groups to deck_table
categories = {
    "mono white": ["mono white", "mono-white", "white weenie", "weenie white", "ww"],
    "mono blue": ["mono blue", "mono-blue"],
    "mono black": ["mono black", "mono-black"],
    "mono red": ["mono red", "mono-red", "red deck wins"],
    "mono green": ["mono green", "mono-green", "monogreen"],
    "azorius": ["azorius", "uw", "wu"],
    "orzhov": ["orzhov", "wb", "bw"],
    "boros": ["boros", "wr", "rw", "winota"],
    "selesnya": ["selesnya", "wg", "gw"],
    "dimir": ["dimir", "ub"],
    "izzet": ["izzet", "ur"],
    "simic": ["simic", "gu"],
    "rakdos": ["rakdos", "br", "rb"],
    "golgari": ["golgari", "bg", "gb"],
    "gruul": ["gruul", "rg"],
    "bant": ["bant", "gwu", "guw", "ugw", "uwg", "wgu", "wug"],
    "esper": ["esper", "wub"],
    "grixis": ["grixis"],
    "jund": ["jund"],
    "naya": ["naya"],
    "jeskai": ["jeskai"],
    "sultai": ["sultai"],
    "mardu": ["mardu", "kroxa doom"],
    "temur": ["temur"],
    "abzan": ["abzan"],
    "4-color": ["omnath", "4c"],
    "colorless": ["forsaken monument"],
    "5-color": ["5c"]
}

for category in categories:
    for placeholder in categories[category]:
        deck_table.loc[
            (deck_table.name.str.lower().str.contains(placeholder)),
            "category"
        ] = category

deck_table.loc[
    (deck_table.name.str.lower() == "gyruda"),
    "category"
] = "4-color"

deck_table.loc[(deck_table.deckId == 360), "category"] = "gruul"

Some codes are not in Scryfall's api, and some (like the core sets) are not picked up in the source code, which is itself a bug. These codes are manually updated to equivalent cards with recognizable card codes.

In [12]:
# Replace all the abu codes with updated codes for basic lands 
# and Disenchant & Unsummon
abu_codes = {
    "092abu": "266eld",
    "170abu": "262eld",
    "130abu": "254eld",
    "196abu": "250eld",
    "249abu": "258eld",
    "065abu": "010znr",
    "271abu": "078m20"
}

# Adding randomly misformed codes.
other_codes = {
    "171cmd": "204m21", # Scavenging Ooze
    "356eld": "101eld", # Rankle, Master of Pranks
    "367eld": "147eld", # Torbran, Thane of Red Fell
    "335eld": "008eld", # Charming Prince
    "346eld": "054eld", # Midnight Clock
    "001unk": "", # Unknown card
    "003ivg": "174grn", # Goblin Electromancer
    "00410m": "004m10", # Baneslayer Angel
    "00514c": "013m21", # Containment Priest
    "021sha": "021shm", # Runed Halo
    "036str": "126eld", # Fling
    "0496th": "032m20", # Pacifism
    "05110m": "051m10", # Essence Scatter
    "05412m": "054m12", # Frost Breath
    "054m12 ": "054m12",
    "079sta": "103m21", # Grim Tutor
    "083urs": "096m21", # Duress
    "10113m": "101m13", # Murder
    "107dar": "241m21", # Tormod's Crypt
    "1136th": "159m21", # Shock
    "12410m": "124m10", # Act of Treason
    "1276th": "171m21", # Volcanic Geyser
    "144gui": "248rna", # Godless Shrine
    "162gui": "257grn", # Steam Vents
    "163gui": "259rna", # Stomping Ground
    "23215m": "245m15", # Radiant Fountain
    "237urs": "063m21", # Rewind
    "309thb": "076thb", # Thryx, the Sudden Storm
    "328iko": "162iko", # Kogla, the Titan Ape
    "334eld": "001eld", # Acclaimed Contender
    "364znr": "215znr"  # Turntimber Symbiosis // Turntimber, Serpentine...
}

card_table["cardId"].replace(to_replace=abu_codes, inplace=True)
deck_list_table["cardId"].replace(to_replace=abu_codes, inplace=True)

deck_list_table["cardId"].replace(to_replace=other_codes, inplace=True)

New decks need to be checked for misformed codes & missing archetypes/categories

In [13]:
deck_table.loc[deck_table.archetype.isna()]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [14]:
deck_table.loc[deck_table.category.isna()]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [15]:
unmatched = set(deck_list_table.loc[~(deck_list_table.cardId.isin(card_table.cardId))]["cardId"])

def find_urls_with_unmatched(unmatched):
    unmatched_deck_ids = list(deck_list_table.loc[deck_list_table.cardId == unmatched]["deckId"])
    return set(deck_table.loc[deck_table.deckId.isin(unmatched_deck_ids)]["deckUrl"])

def print_page_source(unmatched):
    unmatched_url = find_urls_with_unmatched(unmatched).pop()
    print(unmatched_url)
    html = requests.get("https://www.mtgtop8.com/event" + unmatched_url)
    return BeautifulSoup(html.text, features="lxml")

In [16]:
unmatched

{''}

In [17]:
card_table[card_table.name == "Cathartic Reunion"]

,setNumber,setName,name,cmc,color,mana_cost,standardLegality,oracle_text,cardId
587,110,iko,Cathartic Reunion,2,R,{1}{R},legal,"As an additional cost to cast this spell, disc...",110iko
13208,109,kld,Cathartic Reunion,2,R,{1}{R},legal,"As an additional cost to cast this spell, disc...",109kld


In [18]:
events_and_decks = pd.merge(
    event_table, 
    deck_table, 
    on="eventId",
    suffixes=["_event", "_deck"]
).drop_duplicates()

events_decks_pilots = pd.merge(
    events_and_decks,
    pilot_table,
    on="pilotId",
    suffixes=[None, "_pilot"]
).drop_duplicates()

full_table_no_cards = pd.merge(
    events_decks_pilots,
    deck_list_table,
    on="deckId",
    suffixes=[None, "_decklist"]
).drop_duplicates()

full_table = pd.merge(
    full_table_no_cards,
    card_table,
    on="cardId",
    suffixes=[None, "_cards"]
)[[
    "eventId",
    "name_event",
    "date",
    "deckId",
    "pilotId",
    "name_deck",
    "firstName",
    "lastName",
    "cardId",
    "name",
    "count",
    "color",
    "slot",
    "archetype",
    "category"
]].drop_duplicates().sort_values(by=["eventId", "deckId", "slot"]).reset_index(drop=True)

In [24]:
full_table.drop_duplicates("deckId").groupby("archetype")["deckId"].count().sort_values(ascending=False).reset_index()

,archetype,deckId
0,aggro,1052
1,control,550
2,ramp,317
3,mill,148
4,adventure,108
5,rogue,82
6,cycling,31
7,jund,17
8,winota,14
9,midrange,6


In [25]:
full_table.loc[
    full_table["date"] >= "2020-09-25"
].drop_duplicates("deckId").groupby(["archetype"])["deckId"].count().sort_values(ascending=False).reset_index()

,archetype,deckId
0,aggro,945
1,control,451
2,ramp,245
3,mill,146
4,rogue,82
5,adventure,79
6,cycling,29
7,jund,16
8,winota,8
9,midrange,6


In [26]:
full_table.loc[
    full_table["date"] < "2020-09-28"
].drop_duplicates(["deckId"]).groupby("archetype")["deckId"].count().sort_values(ascending=False).reset_index()

,archetype,deckId
0,ramp,127
1,aggro,126
2,control,124
3,adventure,46
4,winota,6
5,cycling,4
6,reclamation,2
7,mill,2
8,flash,2
9,sacrifce,1


In [27]:
full_table.drop_duplicates("deckId").groupby("category")["deckId"].count().sort_values(ascending=False).reset_index()

,category,deckId
0,gruul,455
1,dimir,302
2,4-color,204
3,mono green,196
4,rakdos,146
5,esper,144
6,temur,139
7,mono red,137
8,sultai,107
9,simic,91


In [28]:
full_table.loc[
    full_table["date"] >= "2020-09-28"
].drop_duplicates(["deckId"]).groupby("category")["deckId"].count().sort_values(ascending=False).reset_index()

,category,deckId
0,gruul,431
1,dimir,288
2,mono green,157
3,esper,141
4,rakdos,131
5,temur,119
6,mono red,104
7,simic,87
8,4-color,75
9,azorius,53


In [29]:
full_table.loc[
    full_table["date"] < "2020-09-28"
].drop_duplicates(["deckId"]).groupby("category")["deckId"].count().sort_values(ascending=False).reset_index()

,category,deckId
0,4-color,129
1,sultai,95
2,izzet,40
3,mono green,39
4,mono red,33
5,gruul,24
6,temur,20
7,rakdos,15
8,dimir,14
9,boros,10


In [30]:
full_table.loc[
    full_table["archetype"] == "flash"
]["deckId"].value_counts()

1101    32
1038    30
21      26
Name: deckId, dtype: int64

In [32]:
full_table.isna().any()

eventId       False
name_event    False
date          False
deckId        False
pilotId       False
name_deck     False
firstName     False
lastName      False
cardId        False
name          False
count         False
color         False
slot          False
archetype     False
category      False
dtype: bool

In [33]:
full_table.loc[
    full_table.category.isna()
][["deckId", "name_deck", "category"]]

,deckId,name_deck,category


In [34]:
full_table.loc[
    ~(full_table.name.isin(["Forest", "Swamp", "Mountain", "Island", "Plains"]))
].groupby("name")["count"].sum().sort_values(ascending=False).reset_index().head(30)

,name,count
0,Fabled Passage,6268
1,Bonecrusher Giant // Stomp,4112
2,Lovestruck Beast // Heart's Desire,3294
3,Mystical Dispute,3125
4,Cragcrown Pathway // Timbercrown Pathway,3020
5,Kazandu Mammoth // Kazandu Valley,2765
6,Edgewall Innkeeper,2575
7,"Shatterskull Smashing // Shatterskull, the Ham...",2515
8,Scavenging Ooze,2337
9,Negate,2212


In [44]:
full_table.drop_duplicates(subset=["eventId", "deckId"]).groupby(["name_deck"]).eventId.count().sort_values(ascending=False).reset_index().head(30)

,name_deck,eventId
0,Gruul Aggro,392
1,Mono Green Aggro,191
2,Omnath Ramp,182
3,Esper DOOM Yorion,128
4,Red Deck Wins,128
5,Rakdos Aggro,120
6,Dimir Mill,119
7,Sultai Control,99
8,Dimir Rogue,79
9,Dimir Control,69


### QUESTION 1: 

How has color dominance changed over time?